In [ ]:
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

# api url
base_url = 'http://apis.data.go.kr/1160100/service/GetStockSecuritiesInfoService/getStockPriceInfo?serviceKey=9oUviJPMmodfrNLBPYD0czH%2BOoPnArWcnHsw6T5g2TwnKEs1dwXCp%2FI%2BNfeEGngIAK74J550ZjxJ0CxJy4cK7Q%3D%3D&resultType=json'

# 거래량 함수
def calc_macd(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        contents = response.text  # JSON 데이터 가져오기
    else:
        print(f"API 호출 실패: {response.status_code}")
    
    json_ob = json.loads(contents)

    data = json_ob['response']['body']['items']['item']
    selected_data = []

    for item in data:
        selected_item = {
            'basDt': item['basDt'],
            'itmsNm': item['itmsNm'],
            'mrktCtg': item['mrktCtg'],
            'clpr': item['clpr'],
            'vs': item['vs'],
            'fltRt': item['fltRt'],
            'mkp': item['mkp'],
            'hipr': item['hipr'],
            'lopr': item['lopr']
        }
        selected_data.append(selected_item)
        
    dataframe = pd.DataFrame(selected_data)

    # 데이터 프레임 열 이름 변경
    dataframe = dataframe.rename(columns={
        'basDt': '날짜',
        'itmsNm': '종목명',
        'mrktCtg': '시장구분',
        'clpr': '종가',
        'vs': '거래량',
        'fltRt': '등락률',
        'mkp': '시가총액',
        'hipr': '고가',
        'lopr': '저가'
    })
    
    # 날짜를 날짜 형식으로 변환
    dataframe['날짜'] = pd.to_datetime(dataframe['날짜'], format='%Y%m%d')

    # 종가를 수치로 변환
    dataframe['종가'] = pd.to_numeric(dataframe['종가'])

    # 날짜를 인덱스로 설정
    dataframe.set_index('날짜', inplace=True)
    
    # MACD 계산
    short_window = 12
    long_window = 26
    signal_window = 9

    # 단기(빠른) EMA 계산
    dataframe['ShortEMA'] = dataframe['종가'].ewm(span=short_window, adjust=False).mean()

    # 장기(느린) EMA 계산
    dataframe['LongEMA'] = dataframe['종가'].ewm(span=long_window, adjust=False).mean()

    # MACD Line 계산
    dataframe['MACD'] = dataframe['ShortEMA'] - dataframe['LongEMA']

    # Signal Line 계산
    dataframe['SignalLine'] = dataframe['MACD'].ewm(span=signal_window, adjust=False).mean()

    # MACD Histogram 계산
    dataframe['MACD_Histogram'] = dataframe['MACD'] - dataframe['SignalLine']

    # 시각화
    plt.figure(figsize=(10, 6))

    plt.bar(dataframe.index, dataframe['MACD_Histogram'], label='MACD Histogram', color=np.where(dataframe['MACD_Histogram'] > 0, 'g', 'r'))
    plt.axhline(y=0, color='black', linestyle='--', label='Zero Line')
    plt.title('MACD Histogram')
    plt.xticks(rotation=45)  # x축 레이블에 45도 각도 적용
    plt.xlabel('Date')
    plt.ylabel('MACD Histogram')
    plt.legend()

    return plt.show()